# Step 1: Prompt generator

Create JSONL files for all permutations of all conditions, to be submitted via the OpenAI/vLLM/Anthropic Batch API. Due to 50,000 max prompts/queries per file, it generates 8 versions

Depends on: `input_data/audit_names.xls`

Outputs: 
- For each model, 1 .jsonl and jsonl.zip file to submit to Batch API: `input_data/batch_requests/housing_prompt_v2_{model}.jsonl{|.zip}`
- 1 .csv used to track which task_id corresponds to each prompt: `input_data/housing_prompt_v2.csv`

In [ ]:
import sys
import os
import json
from tqdm import tqdm
import logging
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [ ]:
with open("prefix.txt", "r") as f:
    prefix = f.read()
prefix

In [ ]:
all_models = [
    'gpt-3.5-turbo-0125',
    'gpt-4o-mini-2024-07-18',
    'gpt-4-turbo-2024-04-09',
    'gpt-4o-2024-08-06',
    'gpt-3.5-turbo-1106',
    'inceptionai/jais-family-13b-chat',
    'inceptionai/jais-family-6p7b-chat',
    'inceptionai/jais-family-2p7b-chat', 
    'inceptionai/jais-family-1p3b-chat',
    'CohereForAI/aya-expanse-8b',
    'CohereForAI/aya-expanse-32b',
    'google/gemma-2-27b-it',
    'google/gemma-2-2b-it',
    'google/gemma-2-9b-it',
    'google/gemma-7b-it',
    'google/gemma-2b-it',
    'HuggingFaceTB/SmolLM-1.7B-Instruct',
    'HuggingFaceTB/SmolLM2-1.7B-Instruct',
    'meta-llama/Llama-3.2-3B-Instruct',
    'meta-llama/Llama-3.2-1B-Instruct',
    'meta-llama/Llama-3.1-8B-Instruct',
    'meta-llama/Meta-Llama-3-8B-Instruct',
    'meta-llama/Llama-2-7b-chat-hf',
    'mistralai/Mistral-7B-Instruct-v0.1',
    'mistralai/Mistral-7B-Instruct-v0.3',
    'mistralai/Mistral-Nemo-Instruct-2407', 
    'mistralai/Mistral-Small-Instruct-2409',
    'mistralai/Ministral-8B-Instruct-2410',
    'microsoft/Phi-3-mini-4k-instruct',
    'Qwen/Qwen2.5-0.5B-Instruct',
    'Qwen/Qwen2.5-1.5B-Instruct', 
    'Qwen/Qwen2.5-3B-Instruct',
    'Qwen/Qwen2.5-7B-Instruct', 
    'claude-3-5-sonnet-20241022',
    'claude-3-5-haiku-20241022',
    'claude-3-haiku-20240307'
]

In [ ]:
models = [
    'gpt-3.5-turbo-0125', 
    'gpt-4o-mini-2024-07-18', 
    'gpt-4o-2024-08-06',
    'google/gemma-2-2b-it',
    'meta-llama/Meta-Llama-3-8B-Instruct',
    'meta-llama/Llama-3.2-3B-Instruct',
    'microsoft/Phi-3-mini-4k-instruct'
]

print(len(models))
with open("models.json", "w") as file:
    json.dump(models, file)

In [ ]:
def read_applicants(names_fn="input_data/audit_names.xlsx"):
    """Reads applicant data from the Excel file, matching first and last names by race."""
    try:
        # Load first and last names from the Excel file
        df_first = pd.read_excel(names_fn, sheet_name="first name").fillna(" ")
        df_last = pd.read_excel(names_fn, sheet_name="last name").fillna(" ")
        
        # Group first and last names by race
        first_names_by_race = df_first.groupby('Race')
        last_names_by_race = df_last.groupby('Race')
        
        # Combine first and last names only within the same race
        applicants = []
        for race in first_names_by_race.groups:
            first_names = first_names_by_race.get_group(race)
            last_names = last_names_by_race.get_group(race)
            
            for _, first_row in first_names.iterrows():
                count = 0
                for _, last_row in last_names.iterrows():
                    count += 1
                    full_name = f"{first_row['First Name']} {last_row['Last name']}"
                    applicant = {
                        'Full Name': full_name,
                        'Gender': first_row['Gender'],
                        'Race': last_row['Race'], 
                    }
                    applicants.append(applicant)
                    if count == 2:
                        break
        return applicants
    except Exception as e:
        logging.error(f"Error reading names from audit_names.xlsx: {e}")
        sys.exit(1)

In [ ]:
names_list = read_applicants()
names_list

In [ ]:
len(names_list)

In [ ]:
occupation = ['doctor',
              'software engineer',
              'accountant',
              'teacher',
              'retail associate',
              'construction worker',
              'food service worker',
              'college student',
              'government worker',
              'unemployed',
              'None-control'
             ]

len(occupation)

In [ ]:
living_status = ['just myself', # alone
                 'my roommate and I', # one roommate
                 'my pet and I', # w pet
                 'my spouse and I', # married
                 'my family with kids', # married w kids
                 'None-control'
                ]

len(living_status)

In [ ]:
row_list = []
run_id = 0

# 3 copies of each prompt for each name x occupation x living_status combination
for name in names_list:
    for job in occupation:
        for status in living_status:
            for x in range(0,3):
                row_list.append({'run_id':run_id,
                                 'name':name['Full Name'],
                                 'gender':name['Gender'],
                                 'race':name['Race'],
                                 'occupation':job,
                                 'living_status': status,
                                 'query_response_raw':np.nan,
                                 'query_response':np.nan})
                run_id += 1

In [ ]:
df = pd.DataFrame(row_list)
df

In [ ]:
all_count = df.groupby(['gender', 'race', 'occupation', 'living_status'])['run_id'].count().reset_index()
assert all_count['run_id'].nunique() == 1

gender_race_count = df.groupby(['gender','race'])['run_id'].count().reset_index()
assert gender_race_count['run_id'].nunique() == 1

race_occupation_count = df.groupby(['race','occupation'])['run_id'].count().reset_index()
assert race_occupation_count['run_id'].nunique() == 1

occupation_status_count = df.groupby(['occupation', 'living_status'])['run_id'].count().reset_index()
assert occupation_status_count['run_id'].nunique() == 1

In [ ]:
df.to_csv(f'input_data/{prefix}.csv', index=False)

## Generate prompts

In [ ]:
df = pd.read_csv(f'input_data/{prefix}.csv')
df

In [ ]:
def generate_prompt_xx(name, occupation, living_status):   
    continue

In [ ]:
print(generate_prompt_landlord('Charlie Andersen', 'unemployed', 'just myself'))

In [ ]:
# check control prompt
print(generate_prompt_landlord('Charlie Andersen', 'None-control', 'None-control'))

In [ ]:
!mkdir -p input_data/batch_requests
!mkdir -p input_data/batch_results

In [ ]:
for version in models:
    print(version)
    tasks = []
    for index, row in df.iterrows():

        task = {
            "custom_id": f"task-{row['run_id']}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                # This is what you would have in your Chat Completions API call
                "model": f"{version}",
                # "temperature": 0.7,
                # "max_response_tokens": 64,
                "messages": [
                    {
                        "role": "user",
                        "content": generate_prompt_landlord(row['name'], row['occupation'], row['living_status'])
                    }
                ],
            }
        }

        tasks.append(task)
    version = version.split('/')[-1]
    file_name = f"input_data/batch_requests/{prefix}_{version}.jsonl"
    file_name = file_name
    
    with open(file_name, 'w') as file:
        for obj in tasks:
            file.write(json.dumps(obj) + '\n')

In [ ]:
len(models)

In [ ]:
!head -n 1 input_data/batch_requests/*.jsonl

In [ ]:
%%bash
rm input_data/batch_requests/*.zip
for file in input_data/batch_requests/*.jsonl; do
    if [ -f "$file" ]; then
        zip "${file}.zip" "$file"
    fi
done
ls input_data/batch_requests/*.zip